In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.optimize import fmin_cg, minimize
import sys
sys.path.insert(0,'/home/zahra/PIPELINE')
import scipy as sp
from LogCal_code import Logcal_solutions

#CorrCal_notebook_perturbed_array_file is the same as the CorrCal notebook posted on this account
from CorrCal_notebook_perturbed_array_file import m, S_max, Ndish, Nbls,hist_rel_err_mean_std, Vis_bright_sources, Vis_poisson

In [ ]:
Vis_total = Vis_poisson + Vis_bright_sources

In [ ]:
def Measured_vis(sigma, visibilities):
    '''
    Calculates visibilities with noise
    
    Parameters
    ----------
    sigma : float
        The detector noise
    visibilities : 
        Noiseless visibilities (Nbls,)
        
    Returns
    -------
    Measured visibilities : ndarray
        Visibilities with noise (Nbls,) in units [Jy]
    '''

    mu = 0  
    N_real = np.random.normal(mu, sigma, Nbls)
    N_imag = np.random.normal(mu, sigma, Nbls)
    N_comp = np.array([])
    for i in range(len(N_real)):
        N_comp = np.append(N_comp,complex(N_real[i],N_imag[i]))
    meas_vis = visibilities + N_comp
    return meas_vis

In [ ]:
gain = np.ones(Ndish)
sim_gains = np.ones(2*Ndish)
sim_gains[::2] = gain.real
sim_gains[1::2] = gain.imag

In [ ]:
def recovered_gains(visibilities):
    '''
    Recovered gain amplitudes computed. The phase component is not included here
    
    Parameters
    ----------
    visibilities : 
        Visibilities with noise (Nbls,)
        
    Returns
    -------
    rec_gains : ndarray
        Recovered gain amplitudes (Ndish,)
    '''
    sigma = S_max*0.01
    runs = 500
    rec_gains = np.zeros((runs,Ndish))
    for ind_run in range(runs):
        print (ind_run)
        meas_vis = Measured_vis(sigma, visibilities)    
        rec_gains[ind_run,:] = Logcal_solutions(m,visibilities,visibilities, gain,meas_vis, sigma)[0][:Ndish]  
        print (np.exp(rec_gains[ind_run,:]))
    return rec_gains

In [ ]:
rec_gains = recovered_gains(Vis_total)
#rec_gains = recovered_gains(Vis_poisson)

In [ ]:
std, mean=hist_rel_err_mean_std(np.exp(rec_gains[:]), sim_gains[::2]) # take the exponential
#of recovered gains because LogCal returns logarithm of the gains

plt.hist(mean,'auto') 
plt.xlabel('Mean')
#plt.xscale('log')
plt.show()

In [ ]:
np.median(mean)

In [ ]:
plt.hist(std,'auto') 
plt.xlabel('Standard deviation')
#plt.xscale('log')
plt.show()

In [ ]:
np.median(std)